In [13]:
import nltk

In [14]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pymongo

In [15]:
# Надо выкинуть стоп-слова
stopWords = set(stopwords.words('english'))
wordsFiltered = []

words_info = {}

for i in range(2, 14):
    with open("books/{}.txt".format(i), "r", encoding = "ISO-8859-1") as f:
        text = f.read().lower()
        sentences = nltk.sent_tokenize(text)
        for sent in sentences:
            for (word, role) in nltk.pos_tag(nltk.word_tokenize(sent)):
                if (len(word) > 1) and (word not in stopWords):
                    if word in words_info:
                        words_info[word]["count"] += 1
                    else:
                        words_info[word] = {"count": 1, "role": []}
                    words_info[word]["role"].append((role, sent))

In [16]:
client = pymongo.MongoClient('localhost', 27017)

In [24]:
db = client["lingvabd4"]
posts = db.posts
for i in words_info:
    if not i.isdigit():
        # Так как у нас могут быть такие строки "asdasd." "\dfsf"
        if i[:-1].isalpha() and not i[-1].isalpha():
            new_word = i[:-1]
        elif i[1:].isalpha() and not i[0].isalpha():
            new_word = i[1:]
        else:
            new_word = i
        new_post = {"name": new_word, "info": words_info[i]}
        posts.insert_one(new_post)